In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.6'

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'taylor-swift.txt',
    origin='https://raw.githubusercontent.com/arunavkonwar/collab-files/master/taylor-swift-songs.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

180224/165625 [================================] - 0s 0us/step
('Corpus length:', 165625)


In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

('Number of sequences:', 55189)
('Unique characters:', 57)
Vectorization...


In [0]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [0]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
import random
import sys
import h5py
from google.colab import files

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
model.save_weights('taylor-swift-lyrics-lstm.h5')
files.download('taylor-swift-lyrics-lstm.h5')

('epoch', 1)
Epoch 1/1
55189/55189 [==============================] - 63s 1ms/step - loss: 1.3676
--- Generating with seed: " wish i could run to you
and i hope you know that
everytime "
('------ temperature:', 0.2)
 wish i could run to you
and i hope you know that
everytime i was wind to me to the way you want to the way too what you want to be the way you were the way too what you want to be the way too what you want to be the way too w

hat you want you wilk a bream on the way you want to be the way you want to the way you want to be the way you want to be the way too what you want to tell you when you see you were things the way you want to be the way too what you w()
('------ temperature:', 0.5)
ou were things the way you want to be the way too what you want we were we were the woods think what you were in so me and you can i said and ream oh, oh, oh, oh, oh,oodywood when you say i see you wore the way you want to crouds with

 you are we one here think what you want you and you say when you gottand sorry one what you say you want and when we were why way why's real with to eyes in the will a was a was a time something with the way world with your h()
('------ temperature:', 1.0)
 a was a was a time something with the way world with your ho yeah, fares yead thropiugh
babigh now to place had act line daadin
to do grow timn, i dree he creen, do if the xaly, say, "you say you're a his plakes this cancey, and yoa los mm 

mind whole whan  ot abvering with my all trouble cecieeys (baby our phet onti call the belove the one with anone and you can sie
drew me hea-tet, where you wine you's see (thos trast
thit
that's real you don't strange g()
('------ temperature:', 1.2)
e you's see (thos trast
thit
that's real you don't strange good down
in jreps ep s cryss
when's be re guce afarrline you and and you're live me,
"hen you like and i don't calling vin yer, i'm goital just miss time, she's litelu back i said of whe

n we neat ter, an, go, i'm islonn
frsame
be feet trace all heak in yo
roubye, yea worts you like it highs't heren to belisit overusi, as, the as i rownny trying on, thin, whis wanting's nother
gore
hol'r never goon()
('epoch', 2)
Epoch 1/1
31616/55189 [================>.............] - ETA: 27s - loss: 1.2686

...

asking you
last time i'm asking you
last time i'm asking you
last time i'm asking you
last time i'm asking you
last time i'm asking you
last time i'm asking()
('------ temperature:', 0.5)
i'm asking you
last time i'm asking you
last time i'm asking you
last time i'm asking you
last time i'm asking you
last time i'm asking you
last about if as the time i want your dager, and the starts
i hough a really for soon, on to this isn fashy
i have that say i say yet to see you along like i 

realized what you want, wall, i'm for you say yes, but i said and lonely that i've got and something bad, ahh, hhhy's like i just with you
i love you, i can't liv()
('------ temperature:', 1.0)
ad, ahh, hhhy's like i just with you
i love you, i can't live disce finot lean
like i ss care
back to me that you sound
and say that someday
hey like
and i've been here by i don't know why but don't know i look sorrrou
and lights in the cang
taych to call my fringht be steakin
fixt goodb
i ca

n't say yest gong was a from to remember what the past you unded i'm sirngout on awe
you know you were trouble
i won't make me now eyes but i better holde baby you're ()
('------ temperature:', 1.2)
ble
i won't make me now eyes but i better holde baby you're yet?
are we out of the whastirl on you soury in that veryth, ehd you, but i'll hie tells i've flarlis, babe
am all see this you did
and it lett end that
kind of the babe, this love is breath, ain't oothe story stroudy unssand... 

night
never when take about there when you're forget now, why love it like a bring out
sey they could change bake i can't see it now
and takes about something
but you and ()


MessageError: ignored

In [0]:
files.download('taylor-swift-lyrics-lstm.h5')